# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234858676168                   -0.50    8.0
  2   -7.250160893105       -1.82       -1.39    1.0
  3   -7.250942582268       -3.11       -1.80    4.0
  4   -7.250981424403       -4.41       -1.88    2.0
  5   -7.251320730108       -3.47       -2.48    2.0
  6   -7.251338139644       -4.76       -3.18    1.0
  7   -7.251338659659       -6.28       -3.50    3.0
  8   -7.251338792677       -6.88       -4.08    2.0
  9   -7.251338797693       -8.30       -4.57    4.0
 10   -7.251338798653       -9.02       -5.14    3.0
 11   -7.251338798695      -10.37       -5.44    3.0
 12   -7.251338798701      -11.27       -5.77    1.0
 13   -7.251338798704      -11.41       -6.65    3.0
 14   -7.251338798705      -13.42       -7.01    3.0
 15   -7.251338798705      -14.35       -7.47    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05406551991059068
[ Info: Arnoldi iteration step 2: normres = 0.5879716957962712
[ Info: Arnoldi iteration step 3: normres = 0.8331398491922859
[ Info: Arnoldi iteration step 4: normres = 0.2287781015267912
[ Info: Arnoldi iteration step 5: normres = 0.6131647940669914
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.01e-02, 8.61e-02, 5.63e-01, 2.25e-01, 8.39e-03)
[ Info: Arnoldi iteration step 6: normres = 0.22061004744210516
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.18e-03, 9.35e-02, 1.48e-01, 9.32e-02, 6.31e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09750881488941983
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.47e-04, 9.11e-03, 1.26e-02, 6.80e-02, 6.00e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12413555172017177
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.33e-